In [1]:
!git clone https://github.com/huggingface/search-and-learn

Cloning into 'search-and-learn'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 235 (delta 48), reused 43 (delta 43), pack-reused 160 (from 1)
Receiving objects: 100% (235/235), 87.99 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [2]:
%cd search-and-learn
!pip install -e '.[dev]'

/content/search-and-learn
Obtaining file:///content/search-and-learn
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 93.9 MB/s eta 0:00:00
   ━━━━

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import torch
from vllm import LLM
from sal.config import Config
from sal.models.reward_models import RLHFFlow

config = Config
model_path="meta-llama/Llama-3.2-1B-Instruct"
prm_path="RLHFlow/Llama3.1-8B-PRM-Deepseek-Data"

llm = LLM(
    model=model_path,
    gpu_memory_utilization=0.5,
    enable_prefix_caching=True,
    seed=42,
)

prm = RLHFFlow(prm_path)

/usr/local/lib/python3.10/dist-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


best_of_n
<class 'sal.config.Config'>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

WARNING 12-27 16:16:03 arg_utils.py:953] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-27 16:16:03 config.py:1005] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-27 16:16:03 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, qu

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [3]:
question = 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$'

batch = {
    "problem": [question],
}

In [4]:
from sal.search import beam_search, best_of_n, dvts
from sal.utils.score import score

config.n=32
config.sort_completed= True
config.filter_duplicates= True

result = best_of_n(x=batch, config=config, llm=llm, prm=prm)

In [5]:
from datasets import Dataset

result = Dataset.from_dict(result)

In [6]:
result = score(result, config)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Computing majority & weighted predictions:   0%|          | 0/6 [00:00<?, ?it/s]

Subsample 1:   0%|          | 0/1 [00:00<?, ? examples/s]

Extract answers 1:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute weighted pred 1:   0%|          | 0/1 [00:00<?, ? examples/s]

...

Extract answers 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute weighted pred 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute majority pred 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute naive pred 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Computing majority & weighted predictions: 100%|██████████| 6/6 [00:00<00:00,  6.18it/s]


In [7]:
result['pred'][0]

'<|start_header_id|>assistant<|end_header_id|>\n\n## Step 1: Recall the formulas for converting rectangular coordinates to polar coordinates.\nThe conversion formulas are: $r = \\sqrt{x^2 + y^2}$ and $heta = \\arctan{\\left( \\frac{y}{x} \\right)}$.\n\n## Step 2: Plug in the values of $x$ and $y$ into the formula for $r$.\n$r = \\sqrt{(0)^2 + (3)^2} = \\sqrt{0 + 9} = \\sqrt{9} = 3$\n\n## Step 3: Plug in the values of $x$ and $y$ into the formula for $heta$.\n$heta = \\arctan{\\left( \\frac{(3)}{(0)} \\right)} = \\arctan{(0)} = \\frac{\\pi}{2}$\n\n## Step 4: Write the answer in the required format.\nTherefore, the polar coordinates are $\\left( 3, \\frac{\\pi}{2} \\right)$.\n\nThe final answer is: $\\boxed{\\left( 3, \\frac{\\pi}{2} \\right)}$'

In [8]:
formatted_output = result['pred'][0].replace("<|start_header_id|>assistant<|end_header_id|>\n\n", "").strip()
formatted_output

'## Step 1: Recall the formulas for converting rectangular coordinates to polar coordinates.\nThe conversion formulas are: $r = \\sqrt{x^2 + y^2}$ and $heta = \\arctan{\\left( \\frac{y}{x} \\right)}$.\n\n## Step 2: Plug in the values of $x$ and $y$ into the formula for $r$.\n$r = \\sqrt{(0)^2 + (3)^2} = \\sqrt{0 + 9} = \\sqrt{9} = 3$\n\n## Step 3: Plug in the values of $x$ and $y$ into the formula for $heta$.\n$heta = \\arctan{\\left( \\frac{(3)}{(0)} \\right)} = \\arctan{(0)} = \\frac{\\pi}{2}$\n\n## Step 4: Write the answer in the required format.\nTherefore, the polar coordinates are $\\left( 3, \\frac{\\pi}{2} \\right)$.\n\nThe final answer is: $\\boxed{\\left( 3, \\frac{\\pi}{2} \\right)}$'

In [9]:
from IPython.display import display, Markdown, Latex
display(Markdown(formatted_output))

## Step 1: Recall the formulas for converting rectangular coordinates to polar coordinates.
The conversion formulas are: $r = \sqrt{x^2 + y^2}$ and $heta = \arctan{\left( \frac{y}{x} \right)}$.

## Step 2: Plug in the values of $x$ and $y$ into the formula for $r$.
$r = \sqrt{(0)^2 + (3)^2} = \sqrt{0 + 9} = \sqrt{9} = 3$

## Step 3: Plug in the values of $x$ and $y$ into the formula for $heta$.
$heta = \arctan{\left( \frac{(3)}{(0)} \right)} = \arctan{(0)} = \frac{\pi}{2}$

## Step 4: Write the answer in the required format.
Therefore, the polar coordinates are $\left( 3, \frac{\pi}{2} \right)$.

The final answer is: $\boxed{\left( 3, \frac{\pi}{2} \right)}$

In [17]:
import time

def generate_with_search_and_learn(question, config, llm, prm, method='best_of_n'):
    batch = {"problem": [question]}

    start_time = time.time()
    if method == 'best_of_n':
      result = best_of_n(x=batch, config=config, llm=llm, prm=prm)
    elif method == 'beam_search':
      result = beam_search(examples=batch, config=config, llm=llm, prm=prm)
    elif method == 'dvts':
      result = dvts(examples=batch, config=config, llm=llm, prm=prm)

    elapsed_time = time.time() - start_time
    print(f"\nFinished in {elapsed_time:.2f} seconds\n")

    result = Dataset.from_dict(result)
    result = score(result, config)
    formatted_output = result['pred'][0].replace("<|start_header_id|>assistant<|end_header_id|>\n\n", "").strip()
    return formatted_output


question = 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$'
#question = 'What\'s the capital of Spain?'

config.n=8
config.sort_completed= True
config.filter_duplicates= True
# dvts specific
config.n_beams = config.n // config.beam_width

formatted_output = generate_with_search_and_learn(question=question, config=config, llm=llm, prm=prm, method='beam_search')

Beam search iterations:  48%|████▊     | 19/40 [00:19<00:21,  1.02s/it]


Finished in 19.36 seconds



Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Computing majority & weighted predictions:   0%|          | 0/4 [00:00<?, ?it/s]

Subsample 1:   0%|          | 0/1 [00:00<?, ? examples/s]

...

Extract answers 8:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute weighted pred 8:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute majority pred 8:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute naive pred 8:   0%|          | 0/1 [00:00<?, ? examples/s]

Computing majority & weighted predictions: 100%|██████████| 4/4 [00:00<00:00,  7.81it/s]


In [18]:
display(Markdown(formatted_output))

To convert the point $(0,3)$ from rectangular coordinates to polar coordinates, we use the following formulas:

$r = \sqrt{x^2 + y^2}$

$heta = \tan^{-1} \left(\frac{y}{x}\right)$

First, we calculate $r$:

$r = \sqrt{0^2 + 3^2} = \sqrt{0 + 9} = \sqrt{9} = 3$

Then, we calculate $heta$:

$heta = \tan^{-1} \left(\frac{3}{0}\right)$

Since $\frac{3}{0}$ is undefined, we recognize that the point $(0,3)$ lies on the positive $y$-axis. Therefore, $heta = \frac{\pi}{2}$ since $\tan^{-1}(0) = 0$.

The polar coordinates of the point $(0,3)$ are $\left(3, \frac{\pi}{2}\right)$.

Therefore, the final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$